## clean_animal_services_data_save_to_database
1. loads cleaned animal services data
2. save it as a table in aws postgres database
3. now all three tables, i.e. animal_services, expenses and economic index are ready to be joined

In [8]:

from __future__ import print_function, division


import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

import io

import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [2]:
connection_args = {
    'host': '54.200.221.80', # current aws instance public IP
    'user': 'ubuntu',    
    'dbname': 'proj3',   
    'port': 5432         
}

# We will talk about this magic Python trick!
connection = pg.connect(**connection_args)


In [3]:
import pickle
with open('/home/jieliang//proj3/mvp/mvp_data/eda-6-clean_data_without_dummies.pkl', 'rb') as fp:
    df = pickle.load(fp)

In [4]:
df.drop(['IntakeYear','IntakeMonth', 'OutcomeYear', 'OutcomeMonth', 'IntakeAgeInDays','OutcomeAgeInDays','DaysInShelter'],axis=1,inplace=True)

In [5]:
cur = connection.cursor()

In [6]:
cur.execute("""CREATE TABLE animal_services (
  AnimalType TEXT, 
  IntakeType TEXT, 
  IntakeSubtype TEXT, 
  PrimaryColor TEXT,
  PrimaryBreed TEXT, 
  SecondaryBreed TEXT, 
  Gender TEXT, 
  SecondaryColor TEXT,
  IntakeReason TEXT, 
  IntakeInternalStatus TEXT, 
  IntakeAsilomarStatus TEXT,
  ReproductiveStatusAtIntake TEXT, 
  OutcomeInternalStatus TEXT,
  OutcomeAsilomarStatus TEXT, 
  ReproductiveStatusAtOutcome TEXT,  
  HasIntakeAge INT, 
  HasOutcomeAge INT,
  HasDaysInShelter INT, 
  TimesInShelter INT, 
  ScaledIntakeYear FLOAT,
  ScaledIntakeMonth FLOAT, 
  ScaledOutcomeYear FLOAT, 
  ScaledOutcomeMonth FLOAT,
  ScaledIntakeAgeInDays FLOAT, 
  ScaledOutcomeAgeInDays FLOAT,
  ScaledDaysInShelter FLOAT, 
  intake_year_month TEXT, 
  outcome_year_month TEXT,
  OutcomeType TEXT                          
);""")


In [9]:
output = io.StringIO()
df.to_csv(output, sep='\t', header=False, index=False)
output.seek(0)
contents = output.getvalue()
cur.copy_from(output, 'animal_services', null="") #null values become ''   
connection.commit()

In [14]:
query1 = """CREATE VIEW V1 AS (

  SELECT * FROM
    animal_services a
  LEFT JOIN 
      economic_index e 
  ON
    a.intake_year_month = e.year_month_index
  
);"""

In [15]:
cur.execute(query1)

In [16]:
query2 = """CREATE VIEW V2 AS (

  SELECT * FROM 
    V1
  LEFT JOIN     
    expenses ex
    
  ON
    v1.intake_year_month = ex.year_month
  
);"""

In [17]:
cur.execute(query2)

In [18]:
query3 = 'select * from V2'

In [19]:
joined_df = pd.read_sql_query(query3,con=connection)

In [20]:
joined_df.shape

(141018, 36)

In [21]:
joined_df.head()

,animaltype,intaketype,intakesubtype,primarycolor,primarybreed,secondarybreed,gender,secondarycolor,intakereason,intakeinternalstatus,...,intake_year_month,outcome_year_month,outcometype,date,loiagridx,month,year,year_month_index,year_month,amount
0,CAT,STRAY,OTC,WHITE,DOMESTIC SHORTHAIR,nan,NEUTERED MALE,BROWN,nan,FEARFUL,...,2008-11,2008-11,not_adopted,2008-11-01,-1.57,11.0,2008.0,2008-11,2008-11,30462.52
1,CAT,STRAY,OTC,BLACK,DOMESTIC SHORTHAIR,DOMESTIC SHORTHAIR,UNKNOWN,nan,STRAY,NORMAL,...,2008-11,2008-11,not_adopted,2008-11-01,-1.57,11.0,2008.0,2008-11,2008-11,30462.52
2,OTHER,CONFISCATE,CRUELTY,RED,CHICKEN,nan,MALE,BLACK,nan,OTHER,...,2014-7,2014-7,not_adopted,2014-07-01,7.03,7.0,2014.0,2014-7,2014-7,36847.45
3,OTHER,OTHER,nan,BROWN,OTHER,nan,UNKNOWN,nan,nan,OTHER,...,2014-7,2014-7,not_adopted,2014-07-01,7.03,7.0,2014.0,2014-7,2014-7,36847.45
4,DOG,OWNER SUR,OTC,WHITE,PIT BULL TERRIER,nan,MALE,BROWN,NO TIME,NORMAL,...,2006-9,2006-9,not_adopted,2006-09-01,0.14,9.0,2006.0,2006-9,2006-9,2085.21
